In [1]:
import numpy as np
import netCDF4 as nc
import csv

In [ ]:
def conv_s5p_csv(infile, outfile, optkeys):
    #Load nc file
    nc0  = nc.Dataset(infile, 'r')
    nc0v = nc0['PRODUCT'].variables
    dict0 = {}
    dict0['longitude' ] = nc0v['longitude'][:]
    dict0['latitude' ]  = nc0v['latitude'][:]
    dict0['time_sec']   = nc0v['time'][:] + nc0v['delta_time'][:]*0.001
    dict0['qa_value' ]  = nc0v['qa_value'][:]
    for i in range(len(optkeys)):
        dict0[optkeys[i]] = nc0v[optkeys[i]][:]
    
    nc0.close()
    
    ntime, nsl, ngp = dict0['qa_value'].shape
    time1 = np.zeros(dict0['qa_value'].shape)
    for i in range(ntime):
        for j in range(nsl):
            time1[i,j,:] = dict0['time_sec'][i,j]
    
    mask  = dict0[optkeys[0]].mask
    dict1 = {}
    dict1['longitude' ] = dict0['longitude'][~mask]
    dict1['latitude' ] = dict0['latitude'][~mask]
    dict1['time_sec'] = time1[~mask]
    dict1['qa_value' ] = dict0['qa_value'][~mask]
    for i in range(len(optkeys)):
        dict1[optkeys[i]] = dict0[optkeys[i]][~mask]
    
    nsize = dict1['qa_value'].size
    with open(outfile, 'w') as csv_file:
        #Set Header
        fieldnames = ['time_sec', 'longitude', 'latitude', 'qa_value']
        fieldnames.extend(optkeys)
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        #Write data
        for i in range(nsize):
            dict_tmp = {}
            dict_tmp['time_sec' ] = dict1['time_sec' ][i]
            dict_tmp['longitude'] = dict1['longitude'][i]
            dict_tmp['latitude' ] = dict1['latitude' ][i]
            dict_tmp['qa_value' ] = dict1['qa_value' ][i]
            for j in range(len(optkeys)):
                dict_tmp[optkeys[j]] = dict1[optkeys[j]][i]
            #
            writer.writerow(dict_tmp)
            
            